* https://docs.ragas.io/en/latest/concepts/testset_generation.html
* https://github.com/explodinggradients/ragas/blob/main/src/ragas/testset/generator.py

In [5]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_community.embeddings import HuggingFaceEmbeddings

/Users/I748920/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/I748920/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_ollama import ChatOllama

llm_model1 = ChatOllama(
    model="llama3.1",
    temperature=0 # increase temp for more creative answers
)

llm_model2= ChatOllama(
    model="llama3.1",
    temperature=0 # increase temp for more creative answers
)

embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name, 
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/var/folders/ff/wvby4kh52j96myw15h33jjk40000gn/T/ipykernel_65949/1133824352.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model = HuggingFaceEmbeddings(


In [7]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# # generator with openai models
# generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
# critic_llm = ChatOpenAI(model="gpt-4")
# embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm=llm_model1,
    critic_llm=llm_model2,
    embeddings=embedding_model
)

In [8]:
generator

TestsetGenerator(generator_llm=LangchainLLMWrapper(run_config=RunConfig(timeout=180, max_retries=10, max_wait=60, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42)), critic_llm=LangchainLLMWrapper(run_config=RunConfig(timeout=180, max_retries=10, max_wait=60, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42)), embeddings=<ragas.embeddings.base.LangchainEmbeddingsWrapper object at 0x15fe3e6a0>, docstore=InMemoryDocumentStore(splitter=<langchain_text_splitters.base.TokenTextSplitter object at 0x15fe3e670>, nodes=[], node_embeddings_list=[], node_map={}, run_config=RunConfig(timeout=180, max_retries=10, max_wait=60, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42)))

In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_pdfs(file_paths):
    """
    file_paths must end with .pdf
    PyPDFLoader auto splits the pdf into pages, each page is 1 Document object split by page number
    note that the splitting by page number is not perfect, the actual page number might be +/- 1-2pages.

    returns a dict of key: file_path and value: list of document objects
    """
    documents_dict = {}   
    for f in tqdm(file_paths):
        loader = PyPDFLoader(file_path = f)
        documents = loader.load()
        documents_dict[f] = documents
    return documents_dict

def chunk_list_of_documents(documents):
    """
    input a list of documents as Document objects

    output a list of chunks as Document objects
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 100, # using 20% is a good start
        length_function=len,
        is_separator_regex=False,
        add_start_index=True
    )

    chunks = text_splitter.split_documents(documents)    
    return chunks

In [94]:
import os
import glob
from tqdm import tqdm

# folder_path = "/Users/i748920/Desktop/llms-learning/pdf-chatbot-app/data/short-elements-of-statistical-learning-book"
# os.path.exists(folder_path)
# file_paths = glob.glob(f"{folder_path}/*.pdf")
documents_dict = load_pdfs(
    file_paths=[
        "/Users/i748920/Desktop/llms-learning/pdf-chatbot-app/data/short-chap18-elements-of-statistical-learning-book/chap18 copy.pdf"
    ]
)

100%|██████████| 1/1 [00:00<00:00, 11.31it/s]


In [96]:
len(list(documents_dict.values()))

1

In [98]:
documents = []
for lst in list(documents_dict.values()):
    documents.extend(lst)

len(documents)

3

In [100]:
documents[2]

Document(metadata={'source': '/Users/i748920/Desktop/llms-learning/pdf-chatbot-app/data/short-chap18-elements-of-statistical-learning-book/chap18 copy.pdf', 'page': 2}, page_content='18.2 Nearest Shrunken Centroids 651\nlow bias. When p= 100, we can identify some non-zero coeﬃcients using\nmoderate shrinkage. Finally, when p= 1000, even though there are many\nnonzero coeﬃcients, we don’t have a hope for ﬁnding them and w e need\nto shrink all the way down. As evidence of this, let tj=ˆβj/ˆsej, whereˆβj\nis the ridge regression estimate and ˆsejits estimated standard error. Then\nusing the optimal ridge parameter in each of the three cases, the median\nvalue of|tj|was 2.0, 0.6 and 0.2, and the average number of |tj|values\nexceeding 2 was equal to 9.8, 1.2 and 0.0.\nRidge regression with λ= 0.001 successfully exploits the correlation in\nthe features when p<N, but cannot do so when p≫N. In the latter case\nthere is not enough information in the relatively small numb er of samples\nto eﬃ

In [102]:
documents[0].metadata['source']==documents[0].metadata['filename']

KeyError: 'filename'

In [104]:
for document in documents:
    document.metadata['filename'] = document.metadata['source']+f" - page: {document.metadata['page']}"

In [106]:
documents

[Document(metadata={'source': '/Users/i748920/Desktop/llms-learning/pdf-chatbot-app/data/short-chap18-elements-of-statistical-learning-book/chap18 copy.pdf', 'page': 0, 'filename': '/Users/i748920/Desktop/llms-learning/pdf-chatbot-app/data/short-chap18-elements-of-statistical-learning-book/chap18 copy.pdf - page: 0'}, page_content='This is page 649\nPrinter: Opaque this\n18\nHigh-Dimensional Problems: p≫N\n18.1 When pis Much Bigger than N\nIn this chapter we discuss prediction problems in which the n umber of\nfeaturespis much larger than the number of observations N, often written\np≫N. Such problems have become of increasing importance, espec ially in\ngenomics and other areas of computational biology. We will s ee that high\nvariance and overﬁtting are a major concern in this setting. As a result,\nsimple, highly regularized approaches often become the met hods of choice.\nThe ﬁrst part of the chapter focuses on prediction in both the classiﬁcation\nand regression settings, while th

In [108]:
# generate testset
testset = generator.generate_with_langchain_docs(
    documents=docs, 
    test_size=10, 
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25}
)
#102 because of the 50 docs being split into chunk_size: int = 1024

Filename and doc_id are the same for all nodes.               
Generating: 100%|██████████| 10/10 [1:17:08<00:00, 462.83s/it]


In [109]:
testset

TestDataset(test_data=[DataRow(question='Here is a question that can be fully answered from the given context:\n\n"What does the \'degrees of freedom\' parameter represent in the context of ridge regression?', contexts=['650 18. High-Dimensional Problems: p≫N1.0 1.5 2.0 2.5 3.0Relative error\n20 9 220 featuresTest Error\n1.0 1.5 2.0 2.5 3.0\n99 35 7100 features\n1.0 1.5 2.0 2.5 3.0\n99 87 431000 features\nEffective Degrees of Freedom\nFIGURE 18.1. Test-error results for simulation experiments. Shown are box -\nplots of the relative test errors over 100simulations, for three diﬀerent values\nofp, the number of features. The relative error is the test error d ivided by the\nBayes error, σ2. From left to right, results are shown for ridge regression wit h\nthree diﬀerent values of the regularization parameter λ:0.001,100and1000. The\n(average) eﬀective degrees of freedom in the ﬁt is indicated below each plot.\nvariate regression coeﬃcients1was 9, 33 and 331, respectively, averaged\nover 

In [110]:
test_df = testset.to_pandas()
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,Here is a question that can be fully answered ...,[650 18. High-Dimensional Problems: p≫N1.0 1.5...,The degrees of freedom parameter represents a ...,simple,[{'source': '/Users/i748920/Desktop/llms-learn...,True
1,Here's a question that can be fully answered f...,[18.2 Nearest Shrunken Centroids 651\nlow bias...,Ridge regression successfully exploits the cor...,simple,[{'source': '/Users/i748920/Desktop/llms-learn...,True
2,Here is a question that can be fully answered ...,[650 18. High-Dimensional Problems: p≫N1.0 1.5...,nan,simple,[{'source': '/Users/i748920/Desktop/llms-learn...,True
3,Here is a question that can be fully answered ...,[650 18. High-Dimensional Problems: p≫N1.0 1.5...,Ridge regression outperforms least squares reg...,simple,[{'source': '/Users/i748920/Desktop/llms-learn...,True
4,Here is a question that can be fully answered ...,[650 18. High-Dimensional Problems: p≫N1.0 1.5...,The degrees of freedom parameter represents a ...,simple,[{'source': '/Users/i748920/Desktop/llms-learn...,True
5,"Here is the rewritten question:\n\n""What happe...",[18.2 Nearest Shrunken Centroids 651\nlow bias...,When the number of features exceeds the sample...,reasoning,[{'source': '/Users/i748920/Desktop/llms-learn...,True


In [117]:
test_df.to_csv("sample_ragas_test_set-1.csv",index=False)

In [ ]:
try with openai embeddings